In [25]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv
#from pinecone import Pinecone, ServerlessSpec
import tqdm
load_dotenv()

True

In [26]:
def load_data(data_file):
    loader = DirectoryLoader(data_file,
    glob="*.pdf",
    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [27]:
extracted_data = load_data("data/")

In [28]:
#extracted_data

In [29]:
#Create text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                   chunk_overlap=50)
    
    text_chunks= text_splitter.split_documents(extracted_data)

    return text_chunks
    

In [30]:
text_chunks = text_split(extracted_data=extracted_data)
print(len(text_chunks))

3276


In [31]:
#text_chunks=text_chunks[:1000]

In [32]:
### Download embedding model 

def download_hugging_face_embedding():
    embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [33]:
embeddings=download_hugging_face_embedding()

In [34]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [35]:
query_results=embeddings.embed_query("Hello world")
len(query_results)

384

---

In [36]:
from langchain.vectorstores import FAISS

In [37]:
faiss_db = FAISS.from_documents(text_chunks, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [ ]:
from re import search
retriever = faiss_db.as_retriever(
    search_type="similarity",
    search_kwargs ={'k':4}
)